# **Assignment-2 for CS60075: Natural Language Processing**

#### Instructor : Prof. Sudeshna Sarkar

#### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Dharmin Desai

#### Date of Announcement: 15th Sept, 2021
#### Deadline for Submission: 11.59pm on Wednesday, 22nd Sept, 2021 
#### Submit this .ipynb file, named as `<Your_Roll_Number>_Assn2_NLP_A21.ipynb`

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.  This dataset consists of 50k movie reviews (25k positive, 25k negative). You can download the dataset from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Acads/NLP/Assignment2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Acads/NLP/Assignment2


In [3]:
#Load the IMDB dataset. You can load it using pandas as dataframe
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
print(df.sentiment.unique())
df.head(3)

(50000, 2)
['positive' 'negative']


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [4]:
df.loc[0,'review']

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()
stwords=stopwords.words('english')

def PreProcessing(string):
    result = string.lower()
    result = re.sub('<.*?>',' ',result)
    result = re.sub('http\S+|www.\S+','',result)
    result = re.sub('[^a-z0-9]',' ',result)
    #result = re.sub(r'\.+', ". ", result)
    #result = re.sub('\?', ". ", result)
    result = ' '.join([word for word in result.split() if word not in (stwords)])
    result = ' '.join([stemmer.stem(x) for x in result.split()])
    result = ' '.join([lemmatizer.lemmatize(w) for w in result.split()])   
    result = result.strip(" ")                                                     

    return result

df['preprocessed']=df['review'].apply(PreProcessing)
df.loc[0,'preprocessed']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

In [6]:
df.loc[1,'preprocessed']

'wonder littl product film techniqu unassum old time bbc fashion give comfort sometim discomfort sen realism entir piec actor extrem well chosen michael sheen got polari voic pat truli see seamless edit guid refer william diari entri well worth watch terrif written perform piec master product one great master comedi life realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sen particular scene concern orton halliwel set particular flat halliwel mural decor everi surfac terribl well done'

In [7]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
df['words'] = df["preprocessed"].apply(lambda x: len(x.split(' ')))
print('75% quantile length of sentence',np.quantile(df['words'],0.75))
print('75% quantile length of sentence(positive class)',np.quantile(df.loc[df['sentiment']=='positive','words'],0.75))
print('75% quantile length of sentence(negative class)',np.quantile(df.loc[df['sentiment']=='negative','words'],0.75))
print()

print('avg length of sentence',np.mean(df['words']))
print('avg length of sentence(positive class)',np.mean(df.loc[df['sentiment']=='positive','words']))
print('avg length of sentence(negative class)',np.mean(df.loc[df['sentiment']=='negative','words']))
print()
#we can infer that positive reviews have more number of words being used in the sentence

print('total number samples',len(df))
print('number of positive class',len(df[df['sentiment']=='positive']))
print('number of negative class',len(df[df['sentiment']=='negative']))
df.loc[:1,:]

75% quantile length of sentence 145.0
75% quantile length of sentence(positive class) 148.0
75% quantile length of sentence(negative class) 143.0

avg length of sentence 119.59292
avg length of sentence(positive class) 121.3354
avg length of sentence(negative class) 117.85044

total number samples 50000
number of positive class 25000
number of negative class 25000


,review,sentiment,preprocessed,words
0,One of the other reviewers has mentioned that ...,positive,one review mention watch 1 oz episod hook righ...,163
1,A wonderful little production. <br /><br />The...,positive,wonder littl product film techniqu unassum old...,86


# Naive Bayes classifier

In [8]:
# get reviews column from df
reviews = df['preprocessed']

# get labels column from df
labels = df['sentiment']

In [9]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print(encoded_labels)

[1 1 1 ... 0 0 0]


In [10]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
train_sentences, test_sentences, train_labels, test_labels=train_test_split(reviews,encoded_labels,test_size=0.2)
# train_sentences, test_sentences, train_labels, test_labels
train_sentences.shape, test_sentences.shape, train_labels.shape, test_labels.shape

((40000,), (10000,), (40000,), (10000,))

In [11]:
print("% of +ve sample in train set",train_labels.sum()/len(train_labels)*100)
print("% of +ve sample in test set",test_labels.sum()/len(test_labels)*100)
# thus evenly distributed in labels within the train and test set

% of +ve sample in train set 49.957499999999996
% of +ve sample in test set 50.17


Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [26]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
'''
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
features = vec.get_feature_names()
f_index = vec.vocabulary_ 
d=len(features) #vocab size

print(features)
print(d)

X = X.toarray()
X, X.shape

['000', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '1950', '1950s', '1970', '1970s', '1980', '1st', '20', '2000', '2001', '2005', '2006', '20th', '24', '25', '30', '40', '45', '50', '60', '70', '80', '90', 'abandon', 'abil', 'abl', 'abrupt', 'absolut', 'absorb', 'absurd', 'abus', 'academi', 'accent', 'accept', 'access', 'accid', 'accident', 'accompani', 'accomplish', 'accord', 'account', 'accur', 'accus', 'achiev', 'across', 'act', 'action', 'activ', 'actor', 'actress', 'actual', 'ad', 'adam', 'adapt', 'add', 'addict', 'addit', 'address', 'adequ', 'admir', 'admit', 'adopt', 'ador', 'adult', 'advanc', 'advantag', 'adventur', 'advertis', 'advic', 'advis', 'affair', 'affect', 'afford', 'afraid', 'africa', 'african', 'afternoon', 'afterward', 'age', 'agent', 'ago', 'agre', 'ahead', 'aid', 'aim', 'air', 'airplan', 'aka', 'al', 'ala', 'alan', 'albeit', 'albert', 'alcohol', 'alex', 'alexand', 'alic', 'alien', 'alik', 'aliv', 'allen', 'alli', 'allow', 'almost', 'alon', 'along

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), (40000, 3000))

In [27]:
# Use laplace smoothing for words in test set not present in vocab of train set
# Build the model. Don't use the model from sklearn

#selective filtering of frequencies of samples of positive label
X_1=[]
X_1.extend(X[i]*train_labels[i] for i in range(len(X)))

#selective filtering of frequencies of samples of negative label
X_0=[]
X_0.extend(X[i]*(1-train_labels[i]) for i in range(len(X)))

X_1=np.array(X_1)
X_0=np.array(X_0)

In [28]:
n_w=np.empty([X.shape[1],2])
#frequency of number of words in positive label
n_w[:,0]=X_0.T.sum(axis=1)
#frequency of number of words in negative label
n_w[:,1]=X_1.T.sum(axis=1)

#total number of words in positive label
n_p=n_w[:,1].sum() 
#total number of words in negative label
n_n=n_w[:,0].sum()

In [29]:
p_w=np.empty([X.shape[1],2])
#probablity of word to belong in positive label with laplace smoothening
p_w[:,0]=np.log((n_w[:,0]+1)/(n_n+d)) 
#probablity of word to belong in negative label with laplace smoothening
p_w[:,1]=np.log((n_w[:,1]+1)/(n_p+d))

p_w.sum(axis=0), p_w.max(axis=0)

(array([-26288.38993575, -26133.73415019]), array([-3.74516503, -3.89988813]))

In [30]:
# Test the model on test set and report Accuracy
predict = []

for sent in test_sentences:
    words = sent.split(' ')
    P = N = 0
    for word in words:
        if word in features:
            w_index=f_index[word]
            P+=p_w[w_index,1]
            N+=p_w[w_index,0]
        else :   #for the unknown word (words in test set not in vocab)
            P+=np.log(1/(n_p+d))
            N+=np.log(1/(n_n+d))
    if P>=N:
        predict.append(1)
    else :
        predict.append(0)
        
pred = np.array(predict)

In [31]:
def get_accuracy(true,pred):
  score=0
  for x in range(len(true)):
    if true[x]==pred[x]:
      score+=1
  return score/len(true)*100

In [32]:
get_accuracy(test_labels,pred)

84.17999999999999

# *LSTM* based Classifier

Use the above train and test splits.

In [19]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 150 # choose based on statistics, for example 150 to 200
#as 75% quantile of sentences length varry max of 148
padding_type='post'
trunc_type='post'

In [20]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [21]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [22]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 96s 82ms/step - loss: 0.3726 - accuracy: 0.8343 - val_loss: 0.3411 - val_accuracy: 0.8547
Epoch 2/5
1125/1125 [==============================] - 91s 81ms/step - loss: 0.2734 - accuracy: 0.8905 - val_loss: 0.3262 - val_accuracy: 0.8687
Epoch 3/5
1125/1125 [==============================] - 91s 80ms/step - loss: 0.2344 - accuracy: 0.9084 - val_loss: 0.3197 - val_accuracy: 0.8745
Epoch 4/5
1125/1125 [==============================] - 91s 80ms/step - loss: 0.1987 - accuracy: 0.9232 - val_loss: 0.3338 - val_accuracy: 0.8740
Epoch 5/5
1125/1125 [==============================] - 88s 79ms/step - loss: 0.1656 - accuracy: 0.9371 - val_loss: 0.3531 - val_accuracy: 0.8655


In [23]:
prediction = model.predict(test_padded)
(prediction>0.5).astype('int')

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [24]:
# Calculate accuracy on Test data

prediction = model.predict(test_padded)

# Get probabilities
prediction=(prediction>0.5).astype('int')

# Get labels based on probability 1 if p>= 0.5 else 0
print(classification_report(test_labels,prediction))

# Accuracy : one can use classification_report from sklearn

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      4983
           1       0.88      0.85      0.87      5017

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## Get predictions for random examples

In [25]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
print(model.predict(padded),'\n')

# Get labels based on probability 1 if p>= 0.5 else 0
prediction=model.predict(padded)
pred_prob=(prediction>0.5).astype('int')
pred_prob

[[0.5182307 ]
 [0.37077442]
 [0.31377974]] 



array([[1],
       [0],
       [0]])